In [406]:
import pandas as pd
import utils.DataEngine as de
import matplotlib.pyplot as plt
import collections
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import Ridge, LinearRegression, Lasso
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [407]:
de1 = de.DataEngine('data2024')

In [408]:
df = de1.data['round_1']['price_df']

In [409]:
df

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.0
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.0
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,0.0
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,0.0
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-2,199700,AMETHYSTS,9998,8,9996.0,2.0,9995.0,25.0,10004,2,10005.0,25.0,NaN,NaN,10001.0,0.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0
3997,-2,199800,AMETHYSTS,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,0.0
3998,-2,199900,STARFRUIT,4950,32,NaN,NaN,NaN,NaN,4954,7,4956.0,32.0,NaN,NaN,4952.0,0.0


In [410]:
sdf = df[df['product'] == 'STARFRUIT'].copy()
adf = df[df['product'] == 'AMETHYSTS'].copy()

In [411]:
adf

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.0
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,0.0
5,-2,200,AMETHYSTS,9995,20,NaN,NaN,NaN,NaN,10005,20,NaN,NaN,NaN,NaN,10000.0,0.0
6,-2,300,AMETHYSTS,9996,2,9995.0,29.0,NaN,NaN,10004,2,10005.0,29.0,NaN,NaN,10000.0,0.0
8,-2,400,AMETHYSTS,9996,2,9995.0,25.0,NaN,NaN,10004,2,10005.0,25.0,NaN,NaN,10000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,-2,199500,AMETHYSTS,10002,1,9996.0,1.0,9995.0,22.0,10004,1,10005.0,22.0,NaN,NaN,10003.0,0.0
3992,-2,199600,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10002,7,10004.0,1.0,10005.0,21.0,9999.0,0.0
3995,-2,199700,AMETHYSTS,9998,8,9996.0,2.0,9995.0,25.0,10004,2,10005.0,25.0,NaN,NaN,10001.0,0.0
3997,-2,199800,AMETHYSTS,9995,21,NaN,NaN,NaN,NaN,10002,8,10005.0,21.0,NaN,NaN,9998.5,0.0


In [412]:
sdf

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.0
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,0.0
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,0.0
7,-2,300,STARFRUIT,4996,31,NaN,NaN,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,4999.5,0.0
9,-2,400,STARFRUIT,4995,27,NaN,NaN,NaN,NaN,5002,27,NaN,NaN,NaN,NaN,4998.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,-2,199500,STARFRUIT,4949,23,NaN,NaN,NaN,NaN,4956,23,NaN,NaN,NaN,NaN,4952.5,0.0
3993,-2,199600,STARFRUIT,4949,22,NaN,NaN,NaN,NaN,4956,22,NaN,NaN,NaN,NaN,4952.5,0.0
3994,-2,199700,STARFRUIT,4949,27,NaN,NaN,NaN,NaN,4956,27,NaN,NaN,NaN,NaN,4952.5,0.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0


In [413]:
LEN = 4
X = pd.DataFrame(columns=[f"lagg {i+1}" for i in range(LEN)])
for i in range(LEN):
    X[f"lagg {i+1}"] = sdf.mid_price.shift(i+1)
X = sm.add_constant(X)
y = sdf.mid_price.iloc[4:]
X = X.iloc[4:]

In [414]:
X

,const,lagg 1,lagg 2,lagg 3,lagg 4
9,1.0,4999.5,5000.0,5000.0,5002.5
11,1.0,4998.5,4999.5,5000.0,5000.0
12,1.0,4999.5,4998.5,4999.5,5000.0
15,1.0,5001.0,4999.5,4998.5,4999.5
17,1.0,5000.0,5001.0,4999.5,4998.5
...,...,...,...,...,...
3990,1.0,4949.5,4951.5,4954.0,4950.5
3993,1.0,4952.5,4949.5,4951.5,4954.0
3994,1.0,4952.5,4952.5,4949.5,4951.5
3996,1.0,4952.5,4952.5,4952.5,4949.5


In [415]:
y

9       4998.5
11      4999.5
12      5001.0
15      5000.0
17      5000.0
         ...  
3990    4952.5
3993    4952.5
3994    4952.5
3996    4952.5
3998    4952.0
Name: mid_price, Length: 1996, dtype: float64

In [416]:
m = sm.OLS(y, X)
res = m.fit()

In [417]:
res.params

const     4.392047
lagg 1    0.330427
lagg 2    0.223498
lagg 3    0.251663
lagg 4    0.193519
dtype: float64

In [418]:
model = AutoReg(sdf['mid_price'], lags=4)
model_fitted = model.fit()
model_fitted.summary()

d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(4)   Log Likelihood               -3467.787
Method:               Conditional MLE   S.D. of innovations              1.375
Date:                Thu, 28 Mar 2024   AIC                           6947.574
Time:                        14:52:49   BIC                           6981.167
Sample:                             4   HQIC                          6959.910
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            4.3920     10.797      0.407      0.684     -16.769      25.553
mid_price.L1     0.3304      0.022     15.050      0.000       0.287       0.373
mid_price.L2     0.2235      0.022      9.943      0.000       0.179       0.268
mid_price.L3     0.2517      0.022     11.196      0.000       0.208       0.296
mid_price.L4     0.1935      0.022      8.816      0.000       0.150       0.237
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0004           -0.0000j            1.0004           -0.0000
AR.2           -0.1964           -1.6336j            1.6454           -0.2690
AR.3           -0.1964           +1.6336j            1.6454            0.2690
AR.4           -1.9081           -0.0000j            1.9081           -0.5000
-----------------------------------------------------------------------------
"""

In [419]:
model2 = AutoReg(sdf['mid_price'], lags=2)
model2_fitted = model2.fit()
model2_fitted.summary()

d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(2)   Log Likelihood               -3622.079
Method:               Conditional MLE   S.D. of innovations              1.483
Date:                Thu, 28 Mar 2024   AIC                           7252.158
Time:                        14:52:49   BIC                           7274.558
Sample:                             2   HQIC                          7260.383
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           15.2278     11.614      1.311      0.190      -7.535      37.991
mid_price.L1     0.5429      0.020     27.251      0.000       0.504       0.582
mid_price.L2     0.4540      0.020     22.778      0.000       0.415       0.493
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0021           +0.0000j            1.0021            0.0000
AR.2           -2.1981           +0.0000j            2.1981            0.5000
-----------------------------------------------------------------------------
"""

In [420]:
model2_fitted.params

const           15.227814
mid_price.L1     0.542945
mid_price.L2     0.453990
dtype: float64

In [421]:
model_lst = []
for i in range(1, 6):
    model = AutoReg(sdf['mid_price'], lags=i)
    model_fitted = model.fit()
    model_lst.append(model_fitted)

d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  

In [422]:
def calc_vals(model_lst):
    i = 1
    v_dict = {}
    for model in model_lst:
        p_arr = np.array(model.params[1:])

        l_arr = np.array([sdf['mid_price'].shift(i+1) for i in range(len(p_arr))])

        p_arr_reshaped = p_arr.reshape(-1, 1)

        s = p_arr_reshaped * l_arr
        s = np.sum(s, axis=0)

        r = s - sdf['mid_price'] + model.params[0]

        v_dict[i] = (r.mean(), r.std())
        i += 1
    
    return v_dict

In [423]:
d = calc_vals(model_lst)

C:\Users\giova\AppData\Local\Temp\ipykernel_25788\1848567116.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r = s - sdf['mid_price'] + model.params[0]


In [424]:
d

{1: (1.196035125088776e-09, 1.6650189485640157),
 2: (7.99054823864999e-10, 1.4831440319447955),
 3: (2.848594790195387e-10, 1.401758021400386),
 4: (6.895026967549073e-12, 1.3753115526938362),
 5: (9.930588014395838e-12, 1.3604543890463183)}

In [425]:
de2 = de.DataEngine('data2023')

In [426]:
df1 = de2.data['round_4']['price_df']

In [427]:
bdf = df1[df1['product'] == 'BANANAS']
bdf

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
7,1,3000000,BANANAS,4871.0,9.0,4870.0,23.0,NaN,NaN,4877.0,23.0,NaN,NaN,NaN,NaN,4874.0,0.000000
12,1,3000100,BANANAS,4875.0,1.0,4871.0,4.0,4870.0,22.0,4877.0,22.0,NaN,NaN,NaN,NaN,4876.0,0.000000
23,1,3000200,BANANAS,4870.0,22.0,NaN,NaN,NaN,NaN,4876.0,22.0,NaN,NaN,NaN,NaN,4873.0,1.966309
39,1,3000300,BANANAS,4873.0,1.0,4868.0,26.0,NaN,NaN,4875.0,26.0,NaN,NaN,NaN,NaN,4874.0,8.063477
51,1,3000400,BANANAS,4870.0,8.0,4869.0,28.0,NaN,NaN,4871.0,5.0,4875.0,28.0,NaN,NaN,4870.5,8.017578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329950,3,5999500,BANANAS,4751.0,1.0,4748.0,22.0,NaN,NaN,4755.0,22.0,NaN,NaN,NaN,NaN,4753.0,179236.000000
329963,3,5999600,BANANAS,4748.0,30.0,NaN,NaN,NaN,NaN,4749.0,1.0,4753.0,5.0,4755.0,30.0,4748.5,178564.000000
329977,3,5999700,BANANAS,4748.0,20.0,NaN,NaN,NaN,NaN,4755.0,20.0,NaN,NaN,NaN,NaN,4751.5,178380.000000
329982,3,5999800,BANANAS,4748.0,23.0,NaN,NaN,NaN,NaN,4753.0,2.0,4755.0,23.0,NaN,NaN,4750.5,178496.000000


In [428]:
model4 = AutoReg(bdf['mid_price'], lags=1)
model_fitted4 = model.fit()
model_fitted4.summary()

d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            AutoReg Model Results                             
==============================================================================
Dep. Variable:              mid_price   No. Observations:                 2000
Model:                     AutoReg(5)   Log Likelihood               -3444.381
Method:               Conditional MLE   S.D. of innovations              1.360
Date:                Thu, 28 Mar 2024   AIC                           6902.761
Time:                        14:52:51   BIC                           6941.950
Sample:                             5   HQIC                          6917.152
                                 2000                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.7455     10.691      0.163      0.870     -19.208      22.699
mid_price.L1     0.3017      0.022     13.626      0.000       0.258       0.345
mid_price.L2     0.1868      0.023      8.152      0.000       0.142       0.232
mid_price.L3     0.2189      0.023      9.609      0.000       0.174       0.264
mid_price.L4     0.1468      0.023      6.404      0.000       0.102       0.192
mid_price.L5     0.1454      0.022      6.571      0.000       0.102       0.189
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0001           -0.0000j            1.0001           -0.0000
AR.2            0.3263           -1.5413j            1.5754           -0.2168
AR.3            0.3263           +1.5413j            1.5754            0.2168
AR.4           -1.3311           -0.9991j            1.6643           -0.3975
AR.5           -1.3311           +0.9991j            1.6643            0.3975
-----------------------------------------------------------------------------
"""

In [429]:
model_lst_b = []
for i in range(1, 6):
    model = AutoReg(bdf['mid_price'], lags=i)
    model_fitted = model.fit()
    model_lst_b.append(model_fitted)

d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\GitHub\Prosperity\.env\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  

In [430]:
d1 = calc_vals(model_lst_b)

C:\Users\giova\AppData\Local\Temp\ipykernel_25788\1848567116.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  r = s - sdf['mid_price'] + model.params[0]


In [431]:
d1

{1: (-0.20988451730493118, 1.6666213111172459),
 2: (-0.09743089276677099, 1.483675499343971),
 3: (-0.05567740114250094, 1.4034441614397692),
 4: (-0.033364098768230516, 1.3768540744344433),
 5: (-0.017595422491330216, 1.3624329804958126)}

In [432]:
val4 = bdf['mid_price'].shift(4)
val3 = bdf['mid_price'].shift(3)
val2 = bdf['mid_price'].shift(2)
val1 = bdf['mid_price'].shift(1)

co4, co3, co2, co1 = [-0.01869561,  0.0455032,  0.16316049,  0.8090892]
a = 4.481696494462085

vals = co4*val4 +  co3*val3  +co2*val2 + co1*val1
s = vals - bdf['mid_price'] + a
s.mean(), s.std()

(-0.04512892688968359, 1.5107047407205954)

In [433]:
coef = [0.14543357760079334, 0.14680375449599525, 0.21887676195406236, 0.18681650032375252,  0.3017063656145469]
intercept = 1.7455163186868958

cache = [1, 1, 1, 1, 1]

(np.array(cache) * np.array(coef))

array([0.14543358, 0.14680375, 0.21887676, 0.1868165 , 0.30170637])

In [434]:
sum(coef)

0.9996369599891504

In [435]:
lst_vals = []
for i in range(1, 6):
    len_new = i
    df_test = sdf
    test_var = 'mid_price'
    size = int(len(df_test)/4)

    X_full = pd.DataFrame(columns=[f"lagg {i+1}" for i in range(len_new)])
    t = df_test[test_var].iloc[:size + len_new]
    for i in range(len_new):
        X_full[f"lagg {i+1}"] = t.shift(i+1)

    y_train = t.iloc[len_new:]
    X_train = X_full.iloc[len_new:]

    X_full_k = pd.DataFrame(columns=[f"lagg {i+1}" for i in range(len_new)])
    k = df_test[test_var].iloc[size + len_new:]
    for i in range(len_new):
        X_full_k[f"lagg {i+1}"] = k.shift(i+1)

    y_test = k.iloc[len_new:]
    X_test = X_full_k.iloc[len_new:]

    models_to_test = [LinearRegression()]


    for m in models_to_test:
        m.fit(X_train, y_train)
        y_pred = m.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)

        r2 = r2_score(y_test, y_pred)

        lst_vals.append((len_new, m, mse, ((y_test-y_pred) ** 2).sum(), r2))

lst_vals

[(1,
  LinearRegression(),
  8.119012300830054,
  12162.28042664342,
  0.9435530750462453),
 (2,
  LinearRegression(),
  3.6531030424114768,
  5465.042151447569,
  0.9745912300087181),
 (3,
  LinearRegression(),
  2.5098892652246123,
  3749.774562245571,
  0.9825382234179472),
 (4,
  LinearRegression(),
  2.151963141636818,
  3210.729007322133,
  0.9850260219840844),
 (5,
  LinearRegression(),
  1.925257590230178,
  2868.6338094429652,
  0.9865907497835902)]

In [436]:
def test_regression_models(tot_test, df_test, test_var, models_to_test=[LinearRegression()]):
    lst_vals = []
    size = int(len(df_test)/4)

    for i in range(1, tot_test+1):
        X_full = pd.DataFrame(columns=[f"lagg {j+1}" for j in range(i)])
        t = df_test[test_var].iloc[:size + i]
        for j in range(i):
            X_full[f"lagg {j+1}"] = t.shift(j+1)

        y_train = t.iloc[i:]
        X_train = X_full.iloc[i:]

        X_full_k = pd.DataFrame(columns=[f"lagg {j+1}" for j in range(i)])
        k = df_test[test_var].iloc[size + i:]
        for j in range(i):
            X_full_k[f"lagg {j+1}"] = k.shift(j+1)

        y_test = k.iloc[i:]
        X_test = X_full_k.iloc[i:]

        for m in models_to_test:
            m.fit(X_train, y_train)
            y_pred = m.predict(X_test)

            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            lst_vals.append((i, m, mse, ((y_test - y_pred) ** 2).sum(), r2))

    return lst_vals

In [437]:
test_regression_models(5, sdf, 'bid_price_1'), test_regression_models(5, sdf, 'ask_price_1')[-2][1].coef_

([(1,
   LinearRegression(),
   16.350335045386103,
   24492.801897988382,
   0.8867701147890665),
  (2,
   LinearRegression(),
   7.00970210142644,
   10486.514343733954,
   0.951445664151159),
  (3,
   LinearRegression(),
   4.6281332976845135,
   6914.431146740663,
   0.967931049916585),
  (4,
   LinearRegression(),
   3.721890847781123,
   5553.061144889435,
   0.9742014462431033),
  (5,
   LinearRegression(),
   3.3014502452598524,
   4919.16086543718,
   0.9770870804161549)],
 array([0.27368123, 0.17298578, 0.18998355, 0.15502674, 0.1848425 ]))

In [438]:
val4 = sdf['mid_price'].shift(4)
val3 = sdf['mid_price'].shift(3)
val2 = sdf['mid_price'].shift(2)
val1 = sdf['mid_price'].shift(1)

co4, co3, co2, co1 = [-0.01869561,  0.0455032,  0.16316049,  0.8090892]
a = 4.481696494462085

vals = co4*val4 +  co3*val3  +co2*val2 + co1*val1
s = vals - sdf['mid_price'] + a
s.mean(), s.std()

(-0.18483031123709295, 1.5422175716802697)

In [446]:
sdf[sdf['bid_price_1'] < sdf['mid_price'].shift(1)]

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss,selected_series
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,0.0,4997.0
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,0.0,4997.0
7,-2,300,STARFRUIT,4996,31,NaN,NaN,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,4999.5,0.0,4996.0
9,-2,400,STARFRUIT,4995,27,NaN,NaN,NaN,NaN,5002,27,NaN,NaN,NaN,NaN,4998.5,0.0,4995.0
11,-2,500,STARFRUIT,4996,30,NaN,NaN,NaN,NaN,5003,30,NaN,NaN,NaN,NaN,4999.5,0.0,4996.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,-2,199500,STARFRUIT,4949,23,NaN,NaN,NaN,NaN,4956,23,NaN,NaN,NaN,NaN,4952.5,0.0,4949.0
3993,-2,199600,STARFRUIT,4949,22,NaN,NaN,NaN,NaN,4956,22,NaN,NaN,NaN,NaN,4952.5,0.0,4949.0
3994,-2,199700,STARFRUIT,4949,27,NaN,NaN,NaN,NaN,4956,27,NaN,NaN,NaN,NaN,4952.5,0.0,4949.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0,4949.0


In [454]:
condition_bid = (sdf['bid_price_1'] < sdf['mid_price'].shift(1)) | sdf['bid_price_2'].isna()
bid_selected_series = np.where(condition_bid, sdf['bid_price_1'], sdf['bid_price_2'])

condition_ask = (sdf['ask_price_1'] > sdf['mid_price'].shift(1)) | sdf['ask_price_2'].isna()
ask_selected_series = np.where(condition_ask, sdf['ask_price_1'], sdf['ask_price_2'])

sdf['bid_selected_series'] = bid_selected_series
sdf['ask_selected_series'] = ask_selected_series

In [455]:
test_regression_models(5, sdf, 'bid_selected_series'), test_regression_models(5, sdf, 'ask_selected_series')

([(1,
   LinearRegression(),
   1.5507601559881221,
   2323.038713670207,
   0.9891269418954171),
  (2,
   LinearRegression(),
   1.0282144017439674,
   1538.2087450089753,
   0.9927882719905554),
  (3,
   LinearRegression(),
   0.8828382926737703,
   1318.9604092546128,
   0.993804953755478),
  (4,
   LinearRegression(),
   0.8401474136278826,
   1253.4999411328008,
   0.9941024386639045),
  (5,
   LinearRegression(),
   0.8315686396076414,
   1239.0372730153856,
   0.9941590255364472)],
 [(1,
   LinearRegression(),
   2.9050349856679314,
   4351.742408530561,
   0.9797001030560826),
  (2,
   LinearRegression(),
   1.5013241884726207,
   2245.9809859550405,
   0.9895031643234828),
  (3,
   LinearRegression(),
   1.2042293575398997,
   1799.11866016461,
   0.9915777194538165),
  (4,
   LinearRegression(),
   1.1117145131346766,
   1658.6780535969374,
   0.9922203356931185),
  (5,
   LinearRegression(),
   1.0524860020178561,
   1568.2041430066056,
   0.9926295424612968)])

In [385]:
sdf[sdf.timestamp >= 5000]

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
100,-2,5000,STARFRUIT,4997,30,NaN,NaN,NaN,NaN,5004,30,NaN,NaN,NaN,NaN,5000.5,0.0
102,-2,5100,STARFRUIT,4998,7,4997.0,30.0,NaN,NaN,5004,31,NaN,NaN,NaN,NaN,5001.0,0.0
105,-2,5200,STARFRUIT,4998,8,4997.0,27.0,NaN,NaN,5003,1,5004.0,26.0,NaN,NaN,5000.5,0.0
106,-2,5300,STARFRUIT,5002,1,4997.0,21.0,NaN,NaN,5004,21,NaN,NaN,NaN,NaN,5003.0,0.0
108,-2,5400,STARFRUIT,4997,21,NaN,NaN,NaN,NaN,5002,6,5004.0,21.0,NaN,NaN,4999.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3990,-2,199500,STARFRUIT,4949,23,NaN,NaN,NaN,NaN,4956,23,NaN,NaN,NaN,NaN,4952.5,0.0
3993,-2,199600,STARFRUIT,4949,22,NaN,NaN,NaN,NaN,4956,22,NaN,NaN,NaN,NaN,4952.5,0.0
3994,-2,199700,STARFRUIT,4949,27,NaN,NaN,NaN,NaN,4956,27,NaN,NaN,NaN,NaN,4952.5,0.0
3996,-2,199800,STARFRUIT,4949,21,NaN,NaN,NaN,NaN,4956,21,NaN,NaN,NaN,NaN,4952.5,0.0
